In [14]:
import requests
import json
import pandas as pd 
from datetime import datetime 
import os
import sys 
from importlib import reload
import csv


# dir_path = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', 'config'))

# sys.path.append(dir_path)

import connection
connection  = reload(connection)


In [11]:
conn_pg = connection.connect_to_local_postgres()
cur_pg = conn_pg.cursor()
# df = pd.read_csv('../data/exchange_rate_2024-06-03_160000.csv')
# df.head()
# df.shape

Error connecting to local PostgreSQL: connection to server at "172.17.0.2", port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?



AttributeError: 'NoneType' object has no attribute 'cursor'

In [5]:
csv_path = '../data/exchange_rate_2024-06-03_160000.csv'
# print(csv_path)

../data/exchange_rate_2024-06-03_160000.csv


In [28]:
table_name = 'exchange_rates_import'
conn_pg = connection.connect_to_local_postgres()
cur_pg = conn_pg.cursor()
try:
    rates_table = f'''CREATE TABLE IF NOT EXISTS {table_name} (
                        primary_key varchar PRIMARY KEY, 
                        currency VARCHAR(3),
                        rate NUMERIC(16, 8),
                        base VARCHAR(3),
                        date Date,
                        timestamp VARCHAR(16),
                        rate_checker VARCHAR(45) UNIQUE
                        
                    )'''
    cur_pg.execute(rates_table)
    conn_pg.commit()
    print("Table Creation Successful")
except Exception as e:
    conn_pg.rollback()
    print("Error occurred:", e)

Table Creation Successful


In [23]:
# Open the CSV file
with open(csv_path, 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row if it exists
    for row in reader:
        # Insert each row into the PostgreSQL table
        cur_pg.execute(
          '''INSERT INTO exchange_rates_demo (primary_key, currency, rate, base, date, timestamp, rate_checker) 
                              VALUES (%s, %s, %s, %s, %s, %s, %s )
                              ON CONFLICT(rate_checker) 
                              DO UPDATE
                              SET primary_key = EXCLUDED.primary_key,
                              currency = EXCLUDED.currency,
                              rate = EXCLUDED.rate,
                              base = EXCLUDED.base,
                              date = EXCLUDED.date,
                              timestamp = EXCLUDED.timestamp; ''',
            row
        )

# Commit the transaction
conn_pg.commit()

# Close the cursor and connection
# cur_pg.close()
# conn_pg.close()

In [17]:
conn_pg.rollback()